In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random
import logging
import schedule

In [3]:

# Configure logging to store failed messages in a file
logging.basicConfig(filename="failed_messages.log", level=logging.ERROR, 
                    format="%(asctime)s - Failed to send message to %(message)s", datefmt="%Y-%m-%d %H:%M:%S")

# Loading contacts from excel file (assuming 'Name' and 'Phone' columns)
contacts = pd.read_excel("/Users/mac/Desktop/bulk_msg.xlsx")

# Opening WhatsApp Web
driver = webdriver.Chrome()
driver.get("https://web.whatsapp.com/")
input("Scan the QR code and press Enter...")  # Wait for manual login

''

In [ ]:
def send_message(phone, message):
    try:
        url = f"https://web.whatsapp.com/send?phone={phone}&text={message}"
        driver.get(url)
        time.sleep(random.randint(8,12))  # Randomized delay to mimic human behavior

        send_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label, 'Send')]")))

        send_button.click()

        print(f"✅ Message sent to {phone}")
        time.sleep(random.randint(12, 16))  # Delay between messages
    except Exception as e:
        print(f"❌ Failed to send message to {phone}: {e}")
        logging.error(f"{phone} - {e}")  # Log failed message details

In [ ]:
# Sending messages to all contacts
for index, row in contacts.iterrows():
    send_message(row["Phone"], row["Message"])  # Taking message from Excel file

driver.quit()

✅ Message sent to 919417462900
✅ Message sent to 919464536472


In [ ]:
def schedule_messages():
    """Function to schedule messages at specific times."""
    for index, row in contacts.iterrows():
        # Set your desired schedule here (24-hour format)
        time_to_send = "14:30"  # Example: 2:30 PM

        schedule.every().day.at(time_to_send).do(send_message, row["Phone"], row["Message"])
        print(f"⏳ Scheduled message for {row['Name']} at {time_to_send}")

# Schedule messages
schedule_messages()

print("📅 Scheduler is running. Waiting for the scheduled time...")

# Keep running the script to check the schedule
while True:
    schedule.run_pending()
    time.sleep(30)  # Check every 30 seconds